In [2]:
# prerequisite
# pip install -U imbalanced-learn

In [3]:
from pipelines.DataPipeline import DataPipeline
from transformers.Preprocessing import Preprocessing
from sklearn.model_selection import train_test_split

from transformers.Rebalancer import Rebalancer

from utils.DataHelper import DataHelper
from utils.Common import Config
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

from sklearn.ensemble import RandomForestClassifier

import pandas as pd
import matplotlib.pyplot as plt


In [4]:
RAW_DATA_PATH = "../data/raw/KSI.csv"
df = pd.read_csv(RAW_DATA_PATH)

In [5]:
# fill missing values, adding new columns, extracting useful columns
pc = Preprocessing(df, Config.binary_columns, Config.cat_attribs, Config.num_attribs, Config.label)
new_df = pc.getFrame()

In [6]:
# seperate feature and label
X = new_df[Config.cat_attribs + Config.num_attribs+ Config.binary_columns]
Y = new_df[Config.label]

In [7]:
# pass feature to pipeline and convert it to numerical data
X = DataPipeline(Config.num_attribs,Config.cat_attribs).process(X)

In [8]:
X.isna().sum().sum()

0

In [9]:
Y.value_counts().tolist()

[14246, 2201]

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=Config.test_size, stratify=Y)

In [11]:
from imblearn.over_sampling import SMOTE
smote_minority = SMOTE(n_jobs = -1, sampling_strategy = "minority")

X_train_sm, y_train_sm = smote_minority.fit_resample(X_train, y_train)


c:\Users\yvonn\anaconda3\lib\site-packages\imblearn\over_sampling\_smote\base.py:336: FutureWarning: The parameter `n_jobs` has been deprecated in 0.10 and will be removed in 0.12. You can pass an nearest neighbors estimator where `n_jobs` is already set instead.
  warnings.warn(


In [11]:

# Create param grid

from sklearn.model_selection import RandomizedSearchCV


param_grid = {
              'criterion': ['gini', 'entropy'], 
              'min_samples_split':  range(5,50, 5),
              'min_samples_leaf': range(1, 15, 3),
              'max_depth': range(5, 20, 1)
              } 

clf = RandomizedSearchCV(
    estimator=RandomForestClassifier(),
    scoring="accuracy",
    param_distributions=param_grid,
    cv=10,
    n_iter = 7,
    refit=True,
    verbose=3
)

best_clf = clf.fit(X_train_sm, y_train_sm)

best_clf.best_score_

Fitting 10 folds for each of 7 candidates, totalling 70 fits
[CV 1/10] END criterion=entropy, max_depth=7, min_samples_leaf=1, min_samples_split=35;, score=0.631 total time=   0.9s
[CV 2/10] END criterion=entropy, max_depth=7, min_samples_leaf=1, min_samples_split=35;, score=0.757 total time=   0.9s
[CV 3/10] END criterion=entropy, max_depth=7, min_samples_leaf=1, min_samples_split=35;, score=0.862 total time=   0.9s
[CV 4/10] END criterion=entropy, max_depth=7, min_samples_leaf=1, min_samples_split=35;, score=0.861 total time=   0.9s
[CV 5/10] END criterion=entropy, max_depth=7, min_samples_leaf=1, min_samples_split=35;, score=0.864 total time=   0.9s
[CV 6/10] END criterion=entropy, max_depth=7, min_samples_leaf=1, min_samples_split=35;, score=0.853 total time=   0.8s
[CV 7/10] END criterion=entropy, max_depth=7, min_samples_leaf=1, min_samples_split=35;, score=0.853 total time=   0.9s
[CV 8/10] END criterion=entropy, max_depth=7, min_samples_leaf=1, min_samples_split=35;, score=0.87

0.9282794277268425

In [12]:
best_model = best_clf.best_estimator_
best_model.score(X_test,y_test)

0.8772036474164133

In [13]:
best_clf.best_params_
# ('min_samples_split': 40,  'min_samples_leaf': 1,  'max_depth': 16,  'criterion': 'gini')

{'min_samples_split': 40,
 'min_samples_leaf': 1,
 'max_depth': 16,
 'criterion': 'gini'}

In [12]:
import joblib

best_model = RandomForestClassifier(min_samples_split= 40,  min_samples_leaf= 1,  max_depth= 16,  criterion= 'gini')
best_model.fit(X_train_sm, y_train_sm)
joblib.dump(best_model,'../models/best_model_random_forest.pkl')


['../models/best_model_random_forest.pkl']